### RankForecasting-shortterm-paper
basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [3]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    try:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    except:
        acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [4]:
def plot_prob_forecasts_ex(ts_entry, forecast_entry, ts2, outputfile):

    plot_length = int(forecast_entry[0].samples.shape[1] *1.2) 
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    figcnt = len(ts_entry)
    for idx in range(figcnt):
        fig, axs = plt.subplots(1, 1, figsize=(20, 7))
        #ts_entry[idx][-plot_length:].plot(ax=axs)  # plot the time series
        #forecast_entry[idx].plot(prediction_intervals=prediction_intervals, color='g')
        ts_entry[idx].iloc[-plot_length:,0].plot(ax=axs, linewidth=1)  # plot the time series
        
        ts2.iloc[-plot_length:,0].plot(ax=axs, linewidth=1, label='CurRank')  # plot the time series
        
        forecast_entry[idx].copy_dim(0).plot(prediction_intervals=prediction_intervals, color='g')
        
        plt.ylabel('Rank position')
        plt.xlabel('Lap')
        
        laps = range(1, 200, 20)
        
        #axs.set_xticks(laps)
        #axs.set_xticklabels([1+x*30 for x in range(7)])
        #axs.set_xticklabels(laps)
        
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.savefig(outputfile + '-%d.pdf'%idx)
        plt.show()

In [5]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

### test low mode

In [6]:
acc, ret, pret = {}, {}, {}

In [9]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
                if acc[mid] != []:
                    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                else:
                    pret[mid] = []


INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.78}, mae={1.12}, rmse={2.46},r2={0.92}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.78191489 1.12322097 2.4613872  0.91541442]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 133.96it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.78}, mae={1.15}, rmse={2.95},r2={0.89}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.77659574 1.14904956 2.94504652 0.88672822]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 152.88it/s]

0.03934674533084148 0.07320409933184312 0.03580685746393541


In [10]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.78191489, 1.12322097, 2.4613872 , 0.91541442],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.82446809, 0.97378277, 2.1990805 , 0.93248214],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.7393617 , 1.07159353, 2.41942456, 0.9235528 ],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.83510638, 0.95505418, 2.26225173, 0.93316264],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.77659574, 1.20973783, 2.72462759, 0.89635443],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.85106383, 0.87827715, 2.09440131, 0.93875703],
         [0.718085

In [11]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                if acc[mid] != []:
                    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                else:
                    pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.71}, mae={1.56}, rmse={3.49},r2={0.83}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.71}, mae={1.52}, rmse={3.40},r2={0.84}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.74}, mae={1.56}, rmse={3.47},r2={0.83}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.72340426 1.54606742 3.45355023 0.83345733]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 161.85it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.75}, mae={1.20}, rmse={3.04},r2={0.88}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.73}, mae={1.23}, rmse={3.14},r2={0.87}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.74}, mae={1.20}, rmse={3.05},r2={0.88}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.74113475 1.2109907  3.07686982 0.87633264]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 159.70it/s]

0.04142084112035375 0.0752928530421293 0.0451244091025688


In [ ]:
sys.exit(0)

In [12]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.78191489, 1.12322097, 2.4613872 , 0.91541442],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.82446809, 0.97378277, 2.1990805 , 0.93248214],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.7393617 , 1.07159353, 2.41942456, 0.9235528 ],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.83510638, 0.95505418, 2.26225173, 0.93316264],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.77659574, 1.20973783, 2.72462759, 0.89635443],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.85106383, 0.87827715, 2.09440131, 0.93875703],
         [0.718085

In [19]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-pitage'], axis=0))
    

[[0.7712766  1.23932584 2.9481795  0.87864703]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.80496454 1.10001777 2.82318742 0.89590343]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.75       1.24644195 2.99218259 0.8749919 ]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.78546099 1.11115059 2.8610458  0.89308374]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [22]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-pitage'], axis=0))

[[0.76595745 1.28676654 3.02615006 0.87212463]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.7322695  1.15402381 2.7691987  0.89984605]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.7606383  1.43583021 3.22443275 0.85481637]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.69680851 1.36531059 3.13498363 0.8716213 ]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [20]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-pitage'], axis=0))    

[[0.82446809 1.13595506 2.72469632 0.8963492 ]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.85638298 0.99626932 2.58815098 0.91251842]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.85106383 1.00411985 2.43307423 0.91734918]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.86702128 0.89109966 2.342119   0.92836004]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [21]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-pitage'], axis=0)) 

[[0.85106383 0.87827715 2.09440131 0.93875703]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.82446809 0.80760348 1.97883633 0.94886042]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.82446809 0.97378277 2.1990805  0.93248214]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.83510638 0.95505418 2.26225173 0.93316264]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


### conclusion

best model:

  oracle : 
  
     oracle-TIMEDIFF-{year}-inlap-nopitage  ; best mae
     oracle-RANK-{year}-inlap-pitage        ; best top1acc
     
  pitmodel:
  
     pitmodel-RANK-{year}-inlap-nopitage    ; best mae & top1acc (pitage a little bit less)

### test pred models

In [ ]:
mid = 'oracle-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'deepar-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [34]:
mid = 'pitmodel-2019-test'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.78}, mae={1.16}, rmse={2.93},r2={0.89}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.77659574 1.16361698 2.93459224 0.88753098]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 137.08it/s]

0.043922459599105866 0.07895733323561387 0.05508446626845982


In [ ]:
mid = 'currank-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1,model='naive')
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2018'
full_samples,full_tss = ret[mid][0][1], ret[mid][0][2]
testcar = 12
freq = '1min'
start = pd.Timestamp("01-01-2019", freq=freq) 
fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)
index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
ts = pd.DataFrame(index = index, data = full_tss[testcar])

tss2 = np.zeros_like(full_tss[testcar])
tss2[2:] = full_tss[testcar][:-2]
index2 = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))

ts2 = pd.DataFrame(index = index2, data = tss2)

plot_prob_forecasts_ex([ts], [fc], ts2,'output-car12')

In [ ]:
%debug

In [ ]:
ret.keys()

In [31]:
np.mean(acc['pitmodel-RANK-2018-inlap-nopitage'],axis=0)

array([[0.7712766 , 1.23932584, 2.9481795 , 0.87864703],
       [0.71808511, 1.33932584, 3.24896987, 0.85262365]])

In [32]:
np.mean(acc['pitmodel-RANK-2019-inlap-nopitage'],axis=0)

array([[0.80496454, 1.10001777, 2.82318742, 0.89590343],
       [0.73404255, 1.16255107, 3.05012312, 0.87850116]])

In [29]:
pret['pitmodel-RANK-2018-inlap-nopitage']

{'MSE': 7.433081818341665,
 'abs_error': 6349.0,
 'abs_target_sum': 74024.0,
 'abs_target_mean': 14.542421627584766,
 'seasonal_error': 0.12121212121212122,
 'MASE': 6.509095414793535,
 'sMAPE': 0.1280380038798953,
 'MSIS': 140.7532852512412,
 'QuantileLoss[0.1]': 3947.6,
 'Coverage[0.1]': 0.13283142347159207,
 'QuantileLoss[0.5]': 6349.0,
 'Coverage[0.5]': 0.26016328596758065,
 'QuantileLoss[0.9]': 4205.599999999999,
 'Coverage[0.9]': 0.42102991553219293,
 'RMSE': 2.7263678802285036,
 'NRMSE': 0.18747688315245914,
 'ND': 0.08576948016859397,
 'wQuantileLoss[0.1]': 0.05332865016751324,
 'wQuantileLoss[0.5]': 0.08576948016859397,
 'wQuantileLoss[0.9]': 0.05681400626823732,
 'mean_wQuantileLoss': 0.06530404553478152,
 'MAE_Coverage': 0.2505460739906062}

In [33]:
pret['pitmodel-RANK-2019-inlap-nopitage']

{'MSE': 6.629626312300543,
 'abs_error': 5975.0,
 'abs_target_sum': 81867.0,
 'abs_target_mean': 15.07702315972322,
 'seasonal_error': 0.32386363636363635,
 'MASE': 4.798131863921299,
 'sMAPE': 0.12299058549828731,
 'MSIS': 95.23739095432254,
 'QuantileLoss[0.1]': 3689.2,
 'Coverage[0.1]': 0.12652186645930968,
 'QuantileLoss[0.5]': 5975.0,
 'Coverage[0.5]': 0.2398422867051834,
 'QuantileLoss[0.9]': 3797.9999999999995,
 'Coverage[0.9]': 0.40100707260808066,
 'RMSE': 2.574806072755877,
 'NRMSE': 0.17077682016395765,
 'ND': 0.07298423052023403,
 'wQuantileLoss[0.1]': 0.04506333443267739,
 'wQuantileLoss[0.5]': 0.07298423052023403,
 'wQuantileLoss[0.9]': 0.04639231924951445,
 'mean_wQuantileLoss': 0.05481329473414196,
 'MAE_Coverage': 0.26189083571534855}

### save result df

In [ ]:
df = ret['oracle-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_oracle.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')